In [33]:
import numpy as np
from scipy.linalg import inv, eig
import cvxpy as cp



from src.data_driven import Plant, Cloud

from src.constants import (As, Bs) # system model
from src.constants import (T, KEY_RANGE, 
                       GAMMA_RANGE, INPUT_RANGE,
                         INITIAL_STATE_RANGE, MAX_DISTURBANCE)

In [34]:

def print_experiment_parameters():
    """
    Prints the system and experiment parameters.
    """
    print(f"Trajectory length for generating data: {T}")
    print(f"Input range for the system: {INPUT_RANGE}")
    print(f"Initial state range for the system: {INITIAL_STATE_RANGE}")
    print(f"Key range for random matrices F1 and G1: {KEY_RANGE}")
    print(f"Maximum for disturbance: {MAX_DISTURBANCE}")




# System and experiment parameters
print_experiment_parameters()

Trajectory length for generating data: 20
Input range for the system: 5
Initial state range for the system: 2.5
Key range for random matrices F1 and G1: 1
Maximum for disturbance: 0


In [35]:
# initializing plant and cloud 
plant = Plant(As, Bs)
cloud = Cloud()


In [36]:
n = plant.num_states # number of states
m = plant.num_inputs # number of inputs

Kbar = np.zeros((m, n)) # place holder for Kbar
gamma_results = np.zeros((len(GAMMA_RANGE), 1)) # place holder for gamma_result

In [37]:
"""
What we do:
1) collect data (X1, X0, U0) from a system
2) generate the keys F1 and G1 for transforming the collected data 
3) Using the matrices F1 and G1 transform the collected data 
"""


X1, X0, U0, D0 = plant.collecting_trajectories( 
                        T, INPUT_RANGE, INITIAL_STATE_RANGE, MAX_DISTURBANCE)
F1, G1 = plant.key_generation(n, m, KEY_RANGE)
X1_tilde, X0_tilde, V0 = plant.transforming_data(X1, X0, U0, F1, G1)


In [38]:
"""
What Cloud does:
1) receives the transformed data X1, X0, V0
2) solves an optimization to get the controller Kbar
and gamma bar  
"""


bA, bB, bC, bQ = cloud.ellipsoid_parameters(X1_tilde, X0_tilde,
                                                    V0, MAX_DISTURBANCE)

for gam, gamma in enumerate(GAMMA_RANGE):

    try:
        prob, P, Y = cloud.get_controller_cvxpy(bA, bB, bC, gamma)
        if prob.status == "infeasible":
            gamma_results[[gam], [0]] = -1
        else:
            gamma_results[[gam], [0]] = gamma
            Kbar = Y @ inv(P)
    except:
        gamma_results[[gam], [0]] = -1 # for any other reason we get a flag



gamma_bar = np.max(gamma_results, axis=0).reshape(-1,1)
        
                                                                           

In [39]:

K = F1 + (np.eye(m) + G1) @ Kbar
eig_closed_loop = np.abs(eig(As + Bs @ K,  left=False, right=False))


print(f"""gamma bar received from Cloud: {gamma_bar}
      Controller received from Cloud: {Kbar}
      Controller for the system: {K}
      Absolute value of eigenvalues for the closed-loop system: {eig_closed_loop}
      Max eigen value: {np.max(eig_closed_loop)}
       """)


gamma bar received from Cloud: [[0.033]]
      Controller received from Cloud: [[ 1.06343425 -0.07681454  0.67479937 -1.67547003]
 [ 4.39809992  0.45367691  3.63600996 -3.62499725]]
      Controller for the system: [[ 1.01286046 -1.09178629  0.31932924 -1.91349743]
 [ 5.13385276 -0.56140514  3.25197552 -4.01291752]]
      Absolute value of eigenvalues for the closed-loop system: [2.85596598e-06 4.98001224e-01 3.92873195e-01 3.92873195e-01]
      Max eigen value: 0.4980012236788137
       


In [40]:
# save the date; we need it to simulate the bias injection attack case;
#  see the corresponding notebook
np.savez("clean_data_gamma_bar",
          F1 = F1,
          G1 = G1,
          K_bar = Kbar,
          gamma_bar = gamma_bar,
          T = T,
          key_range = KEY_RANGE,
          input_range = INPUT_RANGE,
          initial_state_range = INITIAL_STATE_RANGE
          )
